In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [7]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
import pandas as pd
import numpy as np
import jsonpath
import json
import time
import datetime
import subprocess
import xlwings as xw
import telebot
import hmac, base64, struct, hashlib
from openpyxl import Workbook, load_workbook
from openpyxl import formatting, styles
from openpyxl.styles import Color, PatternFill, Font, Border
from PIL import ImageGrab
import pyperclip
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', None) #显示单元格完整信息
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

day = 0
pages_user = 150
pages_fircharge = 60
with open(r'C:\Users\User\Desktop\SEO\SEO代码新 0903到期.txt','r') as f:
    access_token = f.read()
# 启动控制台
print('启动cmd。。。。')
command = 'java -jar C:\\Users\\User\\Desktop\\mason-excel-0.0.1-SNAPSHOT.jar'
process = subprocess.Popen(command, shell=True)

url = 'http://fundmng.bsportsadmin.com/api/manage/fund/withdraw/record/list/history'
session = requests.session()

# 第一次获取token
submit_url = 'http://fundmng.bsportsadmin.com/api/manage/user/admin/login/submit'
header0 = {
    'Accept':'application/json, text/plain, */*',
    # 'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Connection':'keep-alive',
    'Content-Length':'48',
    'Content-Type':'application/x-www-form-urlencoded',
    'Cookie':'admin-uid=690; admin-token=db76bebda5274c80adaadd40bd794f24',
    'Device_id':'1.0',
    'Gl_version':'2.0',
    'Host':'fundmng.bsportsadmin.com',
    'Language':'zh_CN',
    'Origin':'http://fundmng.bsportsadmin.com',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/login',
    'Sign':'2bc4c378817f47731f0adf450a627d19',
    'Some':'header',
    'Systemid':"",
    'Timestamp':'1692415901000',
    'Token':'-1',
    'Uid':'-1',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'Version':'1.0'
}
def get_google_code(secret):
    key = base64.b32decode(secret, True)
    msg = struct.pack(">Q", int(time.time()) // 30)
    google_code = hmac.new(key, msg, hashlib.sha1).digest()
    # 很多网上的代码不可用，就在于这儿，没有chr字符串
    o = ord(chr(google_code[19])) & 15
    # google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    return '%06d' % google_code

# selenium模拟浏览器,并运行jar包，生成今日数据
# 指定文件夹路径
folder_path = r'C:\Users\User\Desktop\SEO\_0816'
# 指定文件名
file_name = '今日数据.xlsx'
# 判断文件是否存在
file_path = os.path.join(folder_path, file_name)
if os.path.exists(file_path):
    os.remove(file_path)

# handless无界面浏览器设置
url_jar = 'http://localhost:9881/swagger-ui/index.html#/Mason%20Excel%20%E4%BA%A7%E7%94%9F/index2UsingPOST'
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
path = r'C:\Users\User\IdeaProjects\project1\Bsport\SEO日报\chromedriver.exe'
chrome_options.binary_location = path

# path = r'C:\Users\User\IdeaProjects\project1\Bsport\SEO日报\chromedriver.exe'
def fun_jar():
    browser = webdriver.Chrome(options=chrome_options)
    browser.get(url_jar)
    time.sleep(3)
    button = browser.find_element(By.XPATH,'//div[@class="try-out"]/button')
    button.click()
    time.sleep(2)
    input = browser.find_elements(By.XPATH,'//input')
    input[0].send_keys((datetime.datetime.now()+datetime.timedelta(days=day)).strftime('%Y%m%d'))
    time.sleep(2)
    outpath = r'C:\Users\User\Desktop\SEO\_0816'+'\\'
    input[2].send_keys(9999)
    time.sleep(2)
    input[4].send_keys(outpath)
    time.sleep(2)
    input[5].send_keys((datetime.datetime.now()+datetime.timedelta(days=day)).strftime('%Y%m%d'))
    textarea = browser.find_element(By.TAG_NAME,'textarea')
    textarea.clear()
    textarea.send_keys(access_token)
    time.sleep(3)
    Execute = browser.find_element(By.XPATH,'//button[@class="btn execute opblock-control__btn"]')
    Execute.click()
    browser.quit()

# 运行模拟浏览器函数
fun_jar()
print('开始运行jar包：',time.strftime('%H:%M',time.localtime()))
time.sleep(300)
print('jar包运行结束: ',time.strftime('%H:%M',time.localtime()))

for i in range(3):
    if not os.path.exists(file_path):
        process.terminate()
        process.wait()
        fun_jar()
        time.sleep(300)
        continue
    break

# 后续采集会员列表，首充记录、数据处理
# 读取运行jar包的数据，及历史数据
data_today = pd.read_excel(r'C:\Users\User\Desktop\SEO\_0816\今日数据.xlsx')
data_2_today = pd.read_excel(r'C:\Users\User\Desktop\SEO\_0816\今日数据.xlsx','趋势分析')
daili = pd.read_excel(r'C:\Users\User\Desktop\SEO\数据+ip历史.xlsx','代理总表')
his_data  = pd.read_excel(r'C:\Users\User\Desktop\SEO\SEO总表(12点+18点).xlsx','数据(12点)_2')

# 采集会员列表和会员存记录
url_fircharge = 'http://fundmng.bsportsadmin.com/api/manage/data/detail/firstRecharge'
url_user = 'http://fundmng.bsportsadmin.com/api/manage/user/maintain/user/list'
session = requests.session()

# 采集首存报表
dic_fir = dict({'会员名':[], '所属代理':[],'注册时间':[], '交易时间':[], '交易类型':[], '币种':[], '金额':[]})
# 昨天开始时间戳
yesterday = datetime.date.today() + datetime.timedelta(days=day)
yesterday_start_time = int(time.mktime(time.strptime(str(yesterday), '%Y-%m-%d')))
# 昨天结束时间戳
yesterday_end_time = int(time.mktime(time.strptime(str(datetime.date.today()+ datetime.timedelta(days=day+1)), '%Y-%m-%d'))) - 1
print(yesterday_start_time)
print(yesterday_end_time)

#---------------------token----------------------
# 采集token
google_code = get_google_code('64ehnxj6yily5bhv23kgb62ozuh6yuu2')
data0 = {
    'username': 'Marquis',
    'password': 'qwer123456',
    'code': google_code
}
session0 = requests.Session()
response0  =session0.post(url=submit_url,data=data0,headers=header0)
response0.encoding = 'utf-8'
obj0 = json.loads(response0.text)
token = obj0['data']['token']

header = {
    'Accept':'application/json, text/plain, */*',
    # 'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Connection':'keep-alive',
    'Content-Length':'75',
    'Content-Type':'application/x-www-form-urlencoded',
    'Cookie':'admin-token=67c8b1bd1b434f898ed8570a860355b8; admin-uid=690',
    'Device_id':'1.0',
    'Gl_version':'2.0',
    'Host':'fundmng.bsportsadmin.com',
    'Language':'zh_CN',
    'Menuid':'100112',
    'Opeartionmenu':'%u62A5%u8868%u67E5%u8BE2-%u4F1A%u5458%u9996%u5B58%u62A5%u8868',
    'Origin':'http://fundmng.bsportsadmin.com',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/system/report-query/report-first-recharge',
    'Sign':'ca83944852acc68fe114cbc65f1e1d22',
    'Some':'header',
    'Systemid':'54',
    'Timestamp':'1692092554000',
    'Token':token,
    'Uid':'690',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Version':'1.0'
}
for page in range(1,pages_fircharge+1):
    data = {
        'page': page,
        'size': 20,
        'tradeType': 0,
        'isFake': 0,
        'stime': yesterday_start_time*1000,
        'etime': yesterday_end_time*1000+999
    }
    response = session.post(url_fircharge,headers=header,data=data,timeout=300)
    response.encoding = 'utf-8'
    obj = json.loads(response.text)

    for i in obj['data']['list']:
        dic_fir['会员名'].append(i['userName'])
        dic_fir['所属代理'].append(i['parentName'])
        dic_fir['注册时间'].append(time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(i['createTime']//1000)))
        dic_fir['交易时间'].append(time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(i['regTime']//1000)))
        dic_fir['交易类型'].append(i['tradeType'])
        dic_fir['币种'].append(i['coinCode'])
        dic_fir['金额'].append(i['amount'])
firChargeUser = pd.DataFrame(dic_fir)
print('会员首存行列：',firChargeUser.shape)

# 采集会员列表
dic_user = dict({'会员账号':[], '姓名':[],'代理':[], '注册时间':[]})
for page in range(1,pages_user+1):
    data2 = {
        'page':page,
        'size':20,
        'userVip':'0,1,2,3,4,5,6,7,8,9,10,11',
        'status':'0,1,2,4',
        'sortType':'3',
        'sortStr':'descend',
        'searchType':'1',
        'channelId':'34',
        'registeredStartDate':yesterday_start_time*1000,
        'registeredEndDate':yesterday_end_time*1000+999,
    }
    response2 = session.post(url_user,headers=header,data=data2,timeout=300)
    response2.encoding = 'utf-8'
    obj2 = json.loads(response2.text)
    for i in obj2['data']['list']:
        dic_user['会员账号'].append(i['username'])
        if i['reallyName'] !='':
            dic_user['姓名'].append(i['reallyName'])
        else:
            dic_user['姓名'].append('--')

        dic_user['代理'].append(i['parentName'])
        dic_user['注册时间'].append(time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(i['registerDate']//1000)))
user = pd.DataFrame(dic_user)
print('用户列表行列:',user.shape)
# 删除测试账号
user = user[~user['会员账号'].str.contains('test')&~user['会员账号'].str.contains('ceshi')]
print('去重后：',user.shape)

启动cmd。。。。
开始运行jar包： 17:37
jar包运行结束:  17:42
1693238400
1693324799
会员首存行列： (244, 7)
用户列表行列: (836, 4)
去重后： (743, 4)


In [29]:
print('开始处理shuju...')
shuju = pd.DataFrame({'人员':['Paddy', 'Tony', 'Max', 'Martin', 'Zed', 'Hugo', 'Aber', 'DK', 'Ben','当日汇总'],
                      '日期':(datetime.datetime.now()+datetime.timedelta(days=day)).strftime('%Y/%m/%d'),
                      '时间':(datetime.datetime.now()+datetime.timedelta(days=day)).strftime('%H:%M'),
                      '发送IP':0,
                      '接收IP':0,
                      '对比昨天(总IP)':0,
                      '对比前3天均值(总IP)':0,
                      '对比前7天均值(总IP)':0,
                      '对比昨天(接收IP)':0,
                      '对比前3天均值(接收IP)':0,
                      '对比前7天均值(接收IP)':0,
                      '对比昨天(总注册)':0,
                      '对比前3天均值(总注册)':0,
                      '对比前7天均值(总注册)':0,
                      '对比昨天(总开户)':0,
                      '对比前3天均值(总开户)':0,
                      '对比前7天均值(总开户)':0})

shuju.set_index('人员',inplace = True)
data_today['IP']=pd.to_numeric(data_today['IP'],errors='coerce').replace(np.nan,0).astype('int64')
grp=data_today.groupby('网站名(domain)').agg({'IP':sum})
try:
    shuju.loc['Paddy','发送IP']=grp.loc['paddy.com','IP']
except:
    shuju.loc['Paddy','发送IP']=0
try:
    shuju.loc['Paddy','接收IP']=grp.loc['paddy.bty','IP']
except:
    shuju.loc['Paddy','接收IP']=0
try:
    shuju.loc['Tony','发送IP']=grp.loc['tonyb.com','IP']/2
except:
    shuju.loc['Tony', '发送IP'] = 0
try:
    shuju.loc['Tony','接收IP']=grp.loc['tony.bty','IP']
except:
    shuju.loc['Tony', '接收IP'] = 0
try:
    shuju.loc['Max','发送IP']=grp.loc['mulu.com','IP']
except:
    shuju.loc['Max', '发送IP'] = 0
try:
    shuju.loc['Max','接收IP']=grp.loc['max.bty','IP']
except:
    shuju.loc['Max', '接收IP'] = 0
try:
    shuju.loc['Martin','发送IP']=grp.loc['redquan.com','IP']
except:
    shuju.loc['Martin', '发送IP'] = 0
try:
    shuju.loc['Martin','接收IP']=grp.loc['martin.bty','IP']
except:
    shuju.loc['Martin','接收IP']=0
try:
    shuju.loc['Zed','发送IP']=grp.loc['zed.com','IP']
except:
    shuju.loc['Zed', '发送IP'] = 0
try:
    shuju.loc['Zed','接收IP']=grp.loc['zed.bty','IP']
except:
    shuju.loc['Zed', '接收IP'] = 0
try:
    shuju.loc['Hugo','发送IP']=grp.loc['hugo.com','IP']
except:
    shuju.loc['Hugo', '发送IP'] = 0
try:
    shuju.loc['Hugo','接收IP']=grp.loc['hugo.bty','IP']
except:
    shuju.loc['Hugo','接收IP']=0

shuju.loc['Aber','发送IP']=grp.loc['aber.com','IP']/2
shuju.loc['Aber','接收IP']=grp.loc['aber.bty','IP']
shuju.loc['DK','发送IP']=grp.loc['dk.com','IP']/2
shuju.loc['DK','接收IP']=grp.loc['dk.bty','IP']
shuju.loc['Ben','发送IP']=grp.loc['ben.com','IP']/2
shuju.loc['Ben','接收IP']=grp.loc['ben.bty','IP']
shuju.loc['当日汇总','发送IP']=shuju['发送IP'].sum()
shuju.loc['当日汇总','接收IP']=shuju['接收IP'].sum()

shuju['日期'] = pd.to_datetime(shuju['日期'])
shuju.insert(1,'人员2',shuju.index)
shuju['人员2']=shuju['人员2'].str.lower()

# 第1次merge前，重置索引
shuju.reset_index(inplace=True)

merge_user = pd.merge(user,daili,how = 'left',left_on='代理',right_on='代理线')
grpSEO = merge_user.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
grpSEO.rename(columns={'seo变化数据团队':'注册'},inplace=True)
grpSEO.reset_index(inplace=True)
grpSEO['人员2'] = grpSEO['seo变化数据团队'].str.lower()
grpSEO.set_index('seo变化数据团队',inplace=True)

shuju=shuju.merge(grpSEO,on='人员2',how='left')

shuju['注册率(%)'] = round(shuju['注册']/shuju['发送IP']*100,2)

merge_charge = pd.merge(firChargeUser,daili,how='left',left_on='所属代理',right_on='代理线')
grpCHARGE = merge_charge.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
grpCHARGE= grpCHARGE.rename(columns={'seo变化数据团队':'开户'})
grpCHARGE.reset_index(inplace=True)
grpCHARGE['seo变化数据团队']=grpCHARGE['seo变化数据团队'].str.lower()
grpCHARGE= grpCHARGE.rename(columns={'seo变化数据团队':'人员2'})
# 第2次merge
shuju = pd.merge(shuju,grpCHARGE,how='left',on='人员2')
shuju['转化率(%)'] = round(shuju['开户']/shuju['注册']*100,2)

#------------
# his_data  = pd.read_csv(r'C:\Users\User\Desktop\SEO\SEO每日更新_814.csv',encoding='gbk')
his_data['日期']= pd.to_datetime(his_data['日期'])
be_data = his_data[his_data['日期']==(shuju['日期'][0]+datetime.timedelta(days=-1))][:-1]

# shuju.sort_index(inplace=True)
shuju.set_index('人员',inplace = True)
shuju.sort_index(inplace=True)
be_data.set_index('人员',inplace=True)
be_data.sort_index(inplace=True)

be3_data = his_data[his_data['日期']>=(shuju['日期'][0]+datetime.timedelta(days=-3))]
be3_data = be3_data.groupby('人员').mean()[:-1]

be7_data = his_data[his_data['日期']>=(shuju['日期'][0]+datetime.timedelta(days=-7))]
be7_data = be7_data.groupby('人员').mean()[:-1]

shuju['对比昨天(总IP)']=shuju['发送IP']-be_data['总IP']
shuju['对比前3天均值(总IP)']= shuju['发送IP']-be3_data['总IP']
shuju['对比前7天均值(总IP)']= shuju['发送IP']-be7_data['总IP']

shuju['对比昨天(接收IP)']=shuju['接收IP']-be_data['接收IP']
shuju['对比前3天均值(接收IP)']= shuju['接收IP']-be3_data['接收IP']
shuju['对比前7天均值(接收IP)']= shuju['接收IP']-be7_data['接收IP']

shuju['对比昨天(总注册)']=shuju['注册']-be_data['注册']
shuju['对比前3天均值(总注册)']= shuju['注册']-be3_data['注册']
shuju['对比前7天均值(总注册)']= shuju['注册']-be7_data['注册']

shuju['对比昨天(总开户)']=shuju['开户']-be_data['开户']
shuju['对比前3天均值(总开户)']= shuju['开户']-be3_data['开户']
shuju['对比前7天均值(总开户)']= shuju['开户']-be7_data['开户']
print(shuju)

开始处理shuju...
               日期     人员2     时间  发送IP  接收IP  对比昨天(总IP)  对比前3天均值(总IP)  \
人员                                                                      
Aber   2023-08-29    aber  18:13  2231   403     -367.0    376.000000   
Ben    2023-08-29     ben  18:13   328    90     -103.0     58.000000   
DK     2023-08-29      dk  18:13   107  1011       -1.0    -27.000000   
Hugo   2023-08-29    hugo  18:13  1102     0     -155.0    325.666667   
Martin 2023-08-29  martin  18:13  1049     0     -889.0   -481.666667   
Max    2023-08-29     max  18:13   170   145        0.0     48.000000   
Paddy  2023-08-29   paddy  18:13     1    12        0.0      0.666667   
Tony   2023-08-29    tony  18:13   374     0     -118.0     44.000000   
Zed    2023-08-29     zed  18:13  3537   207     -612.0    836.000000   
当日汇总   2023-08-29    当日汇总  18:13  8899  1868        NaN           NaN   

        对比前7天均值(总IP)  对比昨天(接收IP)  对比前3天均值(接收IP)  对比前7天均值(接收IP)  对比昨天(总注册)  \
人员                               

In [30]:
shuju = shuju.iloc[:,:5].join(shuju.iloc[:,-4:]).join(shuju.iloc[:,5:-4])
shuju.fillna(0,inplace=True)

shuju.loc[:,'对比昨天(总IP)':]=shuju.loc[:,'对比昨天(总IP)':].astype('int64')
shuju['注册'] = shuju['注册'].astype('int64')
shuju['开户'] = shuju['开户'].astype('int64')

for i in shuju.iloc[:,5:].columns:
    shuju.loc['当日汇总',i]=sum(shuju[i])

# 重置三个率
shuju.loc['当日汇总','注册率(%)']=round(shuju.loc['当日汇总','注册']/shuju.loc['当日汇总','发送IP']*100,2)
shuju.loc['当日汇总','转化率(%)']=round(shuju.loc['当日汇总','开户']/shuju.loc['当日汇总','注册']*100,2)

shuju.insert(3,'人员',shuju.index)
shuju.drop('人员2',inplace=True,axis=1)
print('shuju处理完成。。。。')
be_data = his_data[his_data['日期']==(shuju['日期'][0]+datetime.timedelta(days=-1))]

shuju处理完成。。。。


,日期,时间,人员,发送IP,接收IP,注册,注册率(%),开户,转化率(%),对比昨天(总IP),对比前3天均值(总IP),对比前7天均值(总IP),对比昨天(接收IP),对比前3天均值(接收IP),对比前7天均值(接收IP),对比昨天(总注册),对比前3天均值(总注册),对比前7天均值(总注册),对比昨天(总开户),对比前3天均值(总开户),对比前7天均值(总开户)
人员,,,,,,,,,,,,,,,,,,,,,
Aber,2023-08-29,18:13,Aber,2231,403,27,1.21,11,40.74,-367,376,327,-59,239,275,-11,13,16,2,-9,-9
Ben,2023-08-29,18:13,Ben,328,90,2,0.61,2,100.00,-103,58,42,9,62,68,-2,0,0,-2,-32,-36
DK,2023-08-29,18:13,DK,107,1011,5,4.67,1,20.00,-1,-27,-8,-256,584,688,1,-2,-4,0,-20,-19
Hugo,2023-08-29,18:13,Hugo,1102,0,31,2.81,8,25.81,-155,325,356,0,-13,-18,-4,16,19,-5,-16,-18
Martin,2023-08-29,18:13,Martin,1049,0,28,2.67,11,39.29,-889,-481,-507,0,-25,-30,-15,11,15,-9,-11,-14
Max,2023-08-29,18:13,Max,170,145,0,0.00,0,0.00,0,48,47,-17,90,103,0,0,0,0,0,0
Paddy,2023-08-29,18:13,Paddy,1,12,0,0.00,1,0.00,0,0,0,3,8,9,0,0,0,1,1,1
Tony,2023-08-29,18:13,Tony,374,0,11,2.94,4,36.36,-118,44,34,0,-4,-5,-4,4,5,0,-49,-38
Zed,2023-08-29,18:13,Zed,3537,207,10,0.28,2,20.00,-612,836,750,-39,119,136,-3,5,6,-2,-31,-34


In [32]:

be_data
shuju

,日期,时间,人员,总IP,接收IP,注册,注册率(%),开户,转化率(%),对比昨天(总IP),对比前3天均值(总IP),对比前7天均值(总IP),对比昨天(接收IP),对比前3天均值(接收IP),对比前7天均值(接收IP),对比昨天(总注册),对比前3天均值(总注册),对比前7天均值(总注册),对比昨天(总开户),对比前3天均值(总开户),对比前7天均值(总开户)
30,2023-08-28,18:51:00,Aber,2598,462,38.0,1.46,9.0,23.68,1123,926,926,443.0,446.0,446.0,36,37,37,-33,-15,-15
31,2023-08-28,18:51:00,Ben,431,81,4.0,0.93,4.0,100.00,256,194,194,79.0,79.0,79.0,2,3,3,4,-46,-46
32,2023-08-28,18:51:00,DK,108,1267,4.0,3.70,1.0,25.00,-145,-10,-10,1262.0,1259.0,1259.0,2,-7,-7,-19,-26,-26
33,2023-08-28,18:51:00,Hugo,1257,0,35.0,2.78,13.0,37.14,601,681,681,-24.0,-24.0,-24.0,31,30,30,-24,-18,-18
34,2023-08-28,18:51:00,Martin,1938,0,43.0,2.22,20.0,46.51,581,508,508,-39.0,-40.0,-40.0,40,40,40,-5,-6,-6
35,2023-08-28,18:51:00,Max,170,162,1.0,0.59,0.0,0.00,76,63,63,161.0,160.0,160.0,0,0,0,0,0,0
36,2023-08-28,18:51:00,Paddy,1,9,0.0,0.00,0.0,0.00,1,1,1,9.0,8.0,8.0,0,0,0,0,0,0
37,2023-08-28,18:51:00,Tony,492,0,15.0,3.05,4.0,26.67,233,203,203,-3.0,-7.0,-7.0,13,12,12,-96,-51,-51
38,2023-08-28,18:51:00,Zed,4149,246,13.0,0.31,4.0,30.77,2089,1817,1817,235.0,234.0,234.0,12,12,12,-50,-43,-43
39,2023-08-28,18:51:00,当日汇总,11144,4454,153.0,1.37,55.0,35.95,4815,4383,4383,2123.0,2115.0,2115.0,136,127,127,-223,-205,-205


,日期,时间,人员,发送IP,接收IP,注册,注册率(%),开户,转化率(%),对比昨天(总IP),对比前3天均值(总IP),对比前7天均值(总IP),对比昨天(接收IP),对比前3天均值(接收IP),对比前7天均值(接收IP),对比昨天(总注册),对比前3天均值(总注册),对比前7天均值(总注册),对比昨天(总开户),对比前3天均值(总开户),对比前7天均值(总开户)
人员,,,,,,,,,,,,,,,,,,,,,
Aber,2023-08-29,18:13,Aber,2231,403,27,1.21,11,40.74,-367,376,327,-59,239,275,-11,13,16,2,-9,-9
Ben,2023-08-29,18:13,Ben,328,90,2,0.61,2,100.00,-103,58,42,9,62,68,-2,0,0,-2,-32,-36
DK,2023-08-29,18:13,DK,107,1011,5,4.67,1,20.00,-1,-27,-8,-256,584,688,1,-2,-4,0,-20,-19
Hugo,2023-08-29,18:13,Hugo,1102,0,31,2.81,8,25.81,-155,325,356,0,-13,-18,-4,16,19,-5,-16,-18
Martin,2023-08-29,18:13,Martin,1049,0,28,2.67,11,39.29,-889,-481,-507,0,-25,-30,-15,11,15,-9,-11,-14
Max,2023-08-29,18:13,Max,170,145,0,0.00,0,0.00,0,48,47,-17,90,103,0,0,0,0,0,0
Paddy,2023-08-29,18:13,Paddy,1,12,0,0.00,1,0.00,0,0,0,3,8,9,0,0,0,1,1,1
Tony,2023-08-29,18:13,Tony,374,0,11,2.94,4,36.36,-118,44,34,0,-4,-5,-4,4,5,0,-49,-38
Zed,2023-08-29,18:13,Zed,3537,207,10,0.28,2,20.00,-612,836,750,-39,119,136,-3,5,6,-2,-31,-34


In [38]:
with open(r'C:\Users\User\Desktop\SEO\截图文件\seo_12.txt','w') as f:
    f.write(f'截止今日12点,   注册:  {shuju.loc["当日汇总","注册"]} ,开户:  {shuju.loc["当日汇总","开户"]}，整体'
            f'转化率 : {shuju.loc["当日汇总","转化率(%)"]}%\n')
    f.write(f"对比昨日12点,   注册:  {be_data.loc[be_data['人员']=='当日汇总','注册'].values[0]} ,开户:  {be_data.loc[be_data['人员']=='当日汇总','开户'].values[0]}，整体转化率 : {be_data.loc[be_data['人员']=='当日汇总','转化率(%)'].values[0]}%\n")
    f.write(f'同比昨日变化,   注册:  {shuju.loc["当日汇总","注册"]-be_data.loc[be_data["人员"]=="当日汇总","注册"].values[0]} ,开户:  {shuju.loc["当日汇总","开户"]-be_data.loc[be_data["人员"]=="当日汇总","开户"].values[0]}，整体转化率 : {round(shuju.loc["当日汇总","转化率(%)"]-be_data.loc[be_data["人员"]=="当日汇总","转化率(%)"].values[0],2)}%\n')
    f.write('\n')
    f.write(f'人员下降指标如下：\n')
    for i in range(9):
        f.write(shuju.iloc[i,:]['人员'])
        f.write(', ')
        f.write(str(list(shuju.iloc[:,11:].iloc[i,:][shuju.iloc[:,11:].iloc[i,:]<0].index))+'\n')
    f.write('\n')
    f.write(f'转化率<30%的人员：{str(list(shuju[:-1].loc[shuju[:-1]["转化率(%)"]<30,:]["人员"]))}')

44

48

48

1

10

...

3

2

73

1

TypeError: '<' not supported between instances of 'str' and 'int'

In [36]:
# 增加行末表头
header_shuju = pd.DataFrame({'日期':'日期',
                             '时间':'时间',
                             '人员':'人员',
                             '发送IP':'总IP',
                             '接收IP':'接收IP',
                             '注册':'注册',
                             '注册率(%)':'注册率(%)',
                             '开户':'开户',
                             '转化率(%)':'转化率(%)',
                             '对比昨天(总IP)':'对比昨天(总IP)',
                             '对比前3天均值(总IP)':'对比前3天均值(总IP)',
                             '对比前7天均值(总IP)':'对比前7天均值(总IP)',
                             '对比昨天(接收IP)':'对比昨天(接收IP)',
                             '对比前3天均值(接收IP)':'对比前3天均值(接收IP)',
                             '对比前7天均值(接收IP)':'对比前7天均值(接收IP)',
                             '对比昨天(总注册)':'对比昨天(总注册)',
                             '对比前3天均值(总注册)':'对比前3天均值(总注册)',
                             '对比前7天均值(总注册)':'对比前7天均值(总注册)',
                             '对比昨天(总开户)':'对比昨天(总开户)',
                             '对比前3天均值(总开户)':'对比前3天均值(总开户)',
                             '对比前7天均值(总开户)':'对比前7天均值(总开户)'},index=[0])
shuju = shuju.append(header_shuju)
app = xw.App(visible=False,add_book=False)
book = app.books.open(r'C:\Users\User\Desktop\SEO\SEO总表(12点+18点).xlsx')

sheet_shuju = book.sheets['数据(12点)_2']
row_shuju = sheet_shuju.used_range.last_cell.row

sheet_shuju['A'+str(row_shuju+1)].options(index=False,header = False).value = shuju
# sheet_ip['A'+str(row_ip+1)].options(index=False,header = False).value = ip_data
book.save()
book.close()

# # 添加条件格式
wb = load_workbook(r'C:\Users\User\Desktop\SEO\SEO总表(12点+18点).xlsx')
ws = wb['数据(12点)_2']
redFill = Font(color='FF0000')
ws.conditional_formatting.add(f'K{row_shuju +1}:U{row_shuju+10}',
                              formatting.rule.CellIsRule(operator='lessThan',
                                                         formula=['0'],
                                                         font=redFill))
wb.save(filename=r'C:\Users\User\Desktop\SEO\SEO总表(12点+18点).xlsx')
wb.close()
# # 保存截图
book2 = app.books.open(r'C:\Users\User\Desktop\SEO\SEO总表(12点+18点).xlsx')
sheet2_shuju = book2.sheets['数据(12点)_2']
# sheet2_ip =  book2.sheets['ip历史']
range_shuju = sheet2_shuju.range(f'A{row_shuju+1}:U{row_shuju+11}')
range_shuju.api.CopyPicture()
img_shuju = ImageGrab.grabclipboard()  # 获取剪贴板的图片数据
img_shuju.save(r'C:\Users\User\Desktop\SEO\截图文件\shuju(12h)-2.png')  # 保存图片
# pyperclip.copy('')
# # 删除行末表头
def delete_row(sheet, row_index):
    range_obj = sheet.range(f'A{row_index}:A{row_index}')
    range_obj.api.EntireRow.Delete()
delete_row(sheet2_shuju,row_shuju+11)
time.sleep(2)
#

book2.save()
book2.close()
app.quit()
# # 发送到群
with open(r'C:\Users\User\Desktop\SEO\截图文件\seo_12.txt','r') as f:
    text = f.read()

bot_DA = telebot.TeleBot("6106076754:AAHjxPSBpyjwpY-lq1iEslUufW46XQvAfr0")
bot_DA.send_photo(-677235937,open(r'C:\Users\User\Desktop\SEO\截图文件\shuju(12h)-2.png','rb'))
bot_DA.send_message(-677235937,text)
bot_DA.stop_polling()
# 查看
print(shuju)

True

                         日期     时间      人员  发送IP  接收IP   注册  注册率(%)  开户  \
Aber    2023-08-29 00:00:00  18:13    Aber  2231   403   27    1.21  11   
Ben     2023-08-29 00:00:00  18:13     Ben   328    90    2    0.61   2   
DK      2023-08-29 00:00:00  18:13      DK   107  1011    5    4.67   1   
Hugo    2023-08-29 00:00:00  18:13    Hugo  1102     0   31    2.81   8   
Martin  2023-08-29 00:00:00  18:13  Martin  1049     0   28    2.67  11   
Max     2023-08-29 00:00:00  18:13     Max   170   145    0     0.0   0   
Paddy   2023-08-29 00:00:00  18:13   Paddy     1    12    0     0.0   1   
Tony    2023-08-29 00:00:00  18:13    Tony   374     0   11    2.94   4   
Zed     2023-08-29 00:00:00  18:13     Zed  3537   207   10    0.28   2   
当日汇总    2023-08-29 00:00:00  18:13    当日汇总  8899  1868  114    1.28  40   
0                        日期     时间      人员   总IP  接收IP   注册  注册率(%)  开户   
0                        日期     时间      人员   总IP  接收IP   注册  注册率(%)  开户   

        转化率(%)  对比昨天(总IP

In [19]:
his_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   日期             40 non-null     datetime64[ns]
 1   时间             40 non-null     object        
 2   人员             40 non-null     object        
 3   总IP            40 non-null     int64         
 4   接收IP           40 non-null     int64         
 5   注册             39 non-null     float64       
 6   注册率(%)         40 non-null     float64       
 7   开户             40 non-null     float64       
 8   转化率(%)         40 non-null     float64       
 9   对比昨天(总IP)      40 non-null     int64         
 10  对比前3天均值(总IP)   40 non-null     int64         
 11  对比前7天均值(总IP)   40 non-null     int64         
 12  对比昨天(接收IP)     10 non-null     float64       
 13  对比前3天均值(接收IP)  10 non-null     float64       
 14  对比前7天均值(接收IP)  10 non-null     float64       
 15  对比昨天(总注册)      40 non-nul